<a href="https://colab.research.google.com/github/onlyabhilash/spark-nlp-ClinicalResearch/blob/main/1_2_ContextualParser_(Rule_Based_NER).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

Saving spark_nlp_for_healthcare_spark_ocr_4790.json to spark_nlp_for_healthcare_spark_ocr_4790.json


In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

     |████████████████████████████████| 212.4 MB 64 kB/s 
     |████████████████████████████████| 142 kB 20.3 MB/s 
     |████████████████████████████████| 198 kB 70.2 MB/s 
     |████████████████████████████████| 188 kB 7.6 MB/s 
     |████████████████████████████████| 95 kB 2.9 MB/s 
     |████████████████████████████████| 66 kB 5.0 MB/s 


In [ ]:
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

spark = sparknlp_jsl.start(license_keys['SECRET'])

# manually start session
# params = {"spark.driver.memory" : "16G",
#           "spark.kryoserializer.buffer.max" : "2000M",
#           "spark.driver.maxResultSize" : "2000M"}

# spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

In [ ]:
spark

In [ ]:
sample_text = """A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to 
presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis 
three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index 
( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting.
Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . 
She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . 
She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was 
significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , 
or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , 
anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin 
( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed 
as blood samples kept hemolyzing due to significant lipemia .
The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior 
to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , 
the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , 
and lipase was 52 U/L .
 β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged 
 and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . 
 The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides 
 to 1400 mg/dL , within 24 hours .
 Twenty days ago.
 Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . 
 At birth the typical boy is growing slightly faster than the typical girl, but the velocities become equal at about 
 seven months, and then the girl grows faster until four years. 
 From then until adolescence no differences in velocity 
 can be detected. 21-02-2020 
21/04/2020
"""
data = spark.createDataFrame([[sample_text]]).toDF("text").cache()

data.show(truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior ...|
+----------------------------------------------------------------------------------------------------+



### Rules

In [ ]:
!mkdir data

In [8]:
gender = '''male,man,male,boy,gentleman,he,him
female,woman,female,girl,lady,old-lady,she,her
neutral,neutral'''

with open('data/gender.csv', 'w') as f:
    f.write(gender)


gender = {
  "entity": "Gender",
  "ruleScope": "sentence", 
  "completeMatchRegex": "true"
}

import json

with open('data/gender.json', 'w') as f:
    json.dump(gender, f)


date = {
  "entity": "Date ",
  "ruleScope": "sentence",
  "regex": "\\d{1,2}[\\/\\-\\:]{1}(\\d{1,2}[\\/\\-\\:]{1}){0,1}\\d{2,4}",
  "valuesDefinition":[],
  "prefix": [],
  "suffix": [],
  "contextLength": 150,
  "context": []
}

with open('data/date.json', 'w') as f:
    json.dump(date, f)


age = {
  "entity": "Age",
  "ruleScope": "sentence",
   "matchScope":"token",
  "regex":"\\s*(0?[1-9]|[1-9][0-9]|[1][1-9][1-9]|200){1,2}[\\s-,]+|(?i)\\b(?:zero|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty)\\b(?=\\s*year)|\\b(?:(?:one|two|three|four|five|six|seven|eight|nine)? hundred(?:\\sand)?\\s)?(?:(?:twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety)[\\s-]?)?\\b(?:one|two|three|four|five|six|seven|eight|nine)?(?=\\syear)",
  "prefix":["age of"],
  "suffix": ["-years-old",
             "years-old",
             "-year-old",
             "-months-old",
             "-month-old",
             "-months-old",
             "-day-old",
             "-days-old",
             "month old",
             "days old",
             "year old",
             "years old", 
             "years",
             "year", 
             "months", 
             "old"
              ],
  "contextLength": 25,
  "context": [],
  "contextException": ["ago"],
  "exceptionDistance": 10
}

with open('data/age.json', 'w') as f:
    json.dump(age, f)


## Pipeline definition

All rule files from the rule folder are added to the pipeline. They will generate different annotation labels that need to be consolidated. 

In [9]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")


In [10]:
!cd data && ls -lt

total 16
-rw-r--r-- 1 root root 780 Apr 15 08:33 age.json
-rw-r--r-- 1 root root 205 Apr 15 08:33 date.json
-rw-r--r-- 1 root root  97 Apr 15 08:33 gender.csv
-rw-r--r-- 1 root root  75 Apr 15 08:33 gender.json


In [24]:
gender_contextual_parser = ContextualParserApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("entity_gender") \
    .setCaseSensitive(True) \
    .setJsonPath("data/gender.json") \
    .setPrefixAndSuffixMatch(True) \
    .setDictionary('data/gender.csv', read_as=ReadAs.TEXT, options={"delimiter":","})
    #.setDictionary("dictionary.tsv", options={"orientation":"vertical"})

In [28]:
age_contextual_parser = ContextualParserApproach() \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("entity_age") \
        .setJsonPath("data/age.json") \
        .setCaseSensitive(True) \
        .setPrefixAndSuffixMatch(True) \

In [30]:
date_contextual_parser = ContextualParserApproach() \
        .setInputCols(["sentence", "token"]) \
        .setOutputCol("entity_date") \
        .setJsonPath("data/date.json") \
        .setCaseSensitive(True) \
        .setPrefixAndSuffixMatch(True) \

In [31]:
parserPipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    gender_contextual_parser,
    age_contextual_parser,
    date_contextual_parser])

empty_data = spark.createDataFrame([[""]]).toDF("text")

parserModel = parserPipeline.fit(empty_data)

light_model = LightPipeline(parserModel)

In [32]:
annotations = light_model.fullAnnotate(sample_text)[0]
annotations.keys()

dict_keys(['document', 'entity_gender', 'token', 'entity_date', 'entity_age', 'sentence'])

In [33]:
print (annotations['entity_gender'])
print (annotations['entity_age'])
print (annotations['entity_date'])

[Annotation(chunk, 14, 19, female, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '0'}), Annotation(chunk, 471, 473, she, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '1'}), Annotation(chunk, 835, 837, her, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '5'}), Annotation(chunk, 1377, 1379, she, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '9'}), Annotation(chunk, 1517, 1519, her, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '10'}), Annotation(chunk, 2141, 2143, her, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '14'}), Annotation(chunk, 2236, 2238, boy, {'field': 'Gender', 'normalized': 'male', 'confidenceValue': '1.00', 'sentence': '15'}), Annotation(chunk, 2284, 2287, girl, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '15'}), A

In [34]:
import random

def get_color():
    r = lambda: random.randint(100,255)
    return '#%02X%02X%02X' % (r(),r(),r())

In [36]:
get_color()

'#93DCEC'

In [37]:
ner_chunks = []
label_color = {}
unified_entities = {'entity':[]}
for ent_name in annotations.keys():
    if "entity" in ent_name and len(annotations[ent_name])>0:
        ner_chunks.append(ent_name)
        label = annotations[ent_name][0].metadata['field']
        label_color[label] = get_color()
        unified_entities['entity'].extend(annotations[ent_name])

In [38]:
unified_entities['entity'].sort(key=lambda x: x.begin, reverse=False)

In [39]:
unified_entities['entity']

[Annotation(chunk, 14, 19, female, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '0'}),
 Annotation(chunk, 471, 473, she, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '1'}),
 Annotation(chunk, 835, 837, her, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '5'}),
 Annotation(chunk, 1377, 1379, she, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '9'}),
 Annotation(chunk, 1517, 1519, her, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '10'}),
 Annotation(chunk, 2141, 2143, her, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '14'}),
 Annotation(chunk, 2236, 2238, boy, {'field': 'Gender', 'normalized': 'male', 'confidenceValue': '1.00', 'sentence': '15'}),
 Annotation(chunk, 2284, 2287, girl, {'field': 'Gender', 'normalized': 'female', 'confidenceValue': '1.00', 'sentence': '1

### Highlighting the entites with html

In [40]:
html_output = ''
pos = 0

for n in unified_entities['entity']:
    if pos < n.begin and pos < len(sample_text):
        white_text = sample_text[pos:n.begin]
        html_output += '<span class="others" style="background-color: white">{}</span>'.format(white_text)
    pos = n.end+1
    html_output += '<span class="entity-wrapper" style="background-color: {}"><span class="entity-name">{} </span><span class="entity-type">[{}]</span></span>'.format(
        label_color[n.metadata['field']],
        n.result,
        n.metadata['field'])

if pos < len(sample_text):
    html_output += '<span class="others" style="background-color: white">{}</span>'.format(sample_text[pos:])

html_output += """</div>"""

In [41]:
from IPython.display import HTML

HTML(html_output)